In [11]:
import requests
import lxml.html
import pandas as pd
import sqlite3
from pandas.io import sql
import os

In [12]:
import re
import string

# url = 'https://news.v.daum.net/v/20200505000102404'
def get_detail(url):
    body = []
    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·]'
    response = requests.get(url)
    root = lxml.html.fromstring(response.content)
    for p in root.xpath('//*[@id="harmonyContainer"]/section/p'):
        if p.text: # 체크
            body.append(re.sub(punc, '', p.text)) #변환 후 저장
    full_body = ' '.join(body)
    
    return full_body

# get_detail('https://news.v.daum.net/v/20200505000102404')

In [13]:
def db_save(NEWS_LIST):
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            NEWS_LIST.to_sql(name = 'NEWS_LIST', con = con, index = False, if_exists='append') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
        print(len(NEWS_LIST), '건 저장완료..')

In [14]:
def db_delete():
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            cur = con.cursor()
            sql = 'DELETE FROM NEWS_LIST'
            cur.execute(sql)
        except Exception as e:
            print(str(e)) 

In [15]:
def db_drop():
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            cur = con.cursor()
            sql = 'DROP TABLE NEWS_LIST'
            cur.execute(sql)
        except Exception as e:
            print(str(e))

In [16]:
def db_select(): #추출
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            query = 'SELECT * FROM NEWS_LIST'
            NEWS_LIST = pd.read_sql(query, con = con)
        except Exception as e:
            print(str(e)) 
        return NEWS_LIST   


In [17]:
db_drop()

In [18]:
response = requests.get('https://news.daum.net/breakingnews/digital?page=1&regDate=20200505') #2020.05.05 1페이지 뉴스 목록
root = lxml.html.fromstring(response.content)
for li in root.xpath('//*[@id="mArticle"]/div[3]/ul/li'):
    a = li.xpath('div/strong/a')[0]
    url = a.get('href')
    print(url, a.text)

https://v.daum.net/v/20200505233813244 "동물착취 말고 채식하세요" [포토뉴스]
https://v.daum.net/v/20200505221207681 중국, 창정 5B 운반로켓 발사 성공.."우주정류장 건설 서막"(종합)
https://v.daum.net/v/20200505220039611 APEC 통상장관, 경제활동 필수인력 이동 원활화 합의
https://v.daum.net/v/20200505213028369 사고난 차 사진 보내면 AI가 예상수리비 계산
https://v.daum.net/v/20200505213024368 LG의 스마트폰 부활, '벨벳'에 건다
https://v.daum.net/v/20200505213019367 타다 "이동수단 선택 제한은 국민 기본권 침해" 헌법소원
https://v.daum.net/v/20200505205510021 중국, 창정 5B 운반로켓 발사 성공.."우주정류장에 활용 기대"
https://v.daum.net/v/20200505204222924 한지민도 '덕분에 챌린지' 동참..다음 차례는?
https://v.daum.net/v/20200505194003301 자영업자에 손 내민 카드사.. '빅데이터'로 코로나19 돌파
https://v.daum.net/v/20200505193959299 사고차량 사진 띄우면 예상수리비 산출 '보험개발원 AOSα'
https://v.daum.net/v/20200505193958298 디지털 신사업 뛰어든 신한아이타스 'KMS' 오픈
https://v.daum.net/v/20200505193936283 현대百면세점 "5월 특수만 기다렸다"
https://v.daum.net/v/20200505193933282 집콕 시대 'TV·컴퓨터' 제일 잘 나가네
https://v.daum.net/v/20200505193931281 "위메프서 사면 100% 최저가"
https://v.daum.net/v/20200505193921280 달아오른 이커머스 패권전쟁.. 

In [19]:
#db_delete()

In [29]:
page = 1
max_page = 0
dt = '20200505'
while(True):
    df_list = []
    response = requests.get('https://news.daum.net/breakingnews/digital?page={}&regDate={}'\
                            .format(page, dt))
    root = lxml.html.fromstring(response.content)
    for li in root.xpath('//*[@id="mArticle"]/div[3]/ul/li'):
        a = li.xpath('div/strong/a')[0]
        url = a.get('href')
        df = pd.DataFrame({'URL' : [url],'TITLE':[a.text],'ARTICLE' : [get_detail(url)]})
        df_list.append(df)   
        
    if df_list:   
        df_10 = pd.concat(df_list) #concat: concatenation(접합)의 줄임말
        db_save(df_10)

    for a in root.xpath('//*[@id="mArticle"]/div[3]/div/span/a'): #페이지 수
        try:
            num = int(a.text) # ex) 6페이지: <a href="/breakingnews/digital?page=6&amp;regDate=20200505" class="num_page">6</a> -> 여기서 6이 텍스트
            if max_page < num:
                max_page = num       
        except:
            pass
    
    span = root.xpath('//*[@id="mArticle"]/div[3]/div/span/a[@class="btn_page btn_next"]') #꺽새가 있는지 없는지 확인/ class지정에 @ 들어감
    #print(span)
    if (len(span) <= 0) & (page > max_page): #span이 0이하 이면 꺽새 없음
        break;
    else:
        page = page + 1

15 건 저장완료..
[<Element a at 0x1e1a27df728>]
15 건 저장완료..
[<Element a at 0x1e1a27e95e8>]
15 건 저장완료..
[<Element a at 0x1e1a28144f8>]
15 건 저장완료..
[<Element a at 0x1e1a2814d68>]
15 건 저장완료..
[<Element a at 0x1e1a2817138>]
15 건 저장완료..
[<Element a at 0x1e1a2814cc8>]
15 건 저장완료..
[<Element a at 0x1e1a27df958>]
15 건 저장완료..
[<Element a at 0x1e1a27df4a8>]
15 건 저장완료..
[<Element a at 0x1e1a284a4a8>]
15 건 저장완료..
[<Element a at 0x1e1a27dfa48>]
15 건 저장완료..
[<Element a at 0x1e1a27dfa98>]
15 건 저장완료..
[<Element a at 0x1e1a27ce0e8>]
15 건 저장완료..
[<Element a at 0x1e1a2818f98>]
15 건 저장완료..
[<Element a at 0x1e1a2817ae8>]
15 건 저장완료..
[<Element a at 0x1e1a27ce4a8>]
15 건 저장완료..
[<Element a at 0x1e1a2867278>]
15 건 저장완료..
[<Element a at 0x1e1a27ce598>]
15 건 저장완료..
[<Element a at 0x1e1a15ac318>]
15 건 저장완료..
[<Element a at 0x1e1a27cef48>]
15 건 저장완료..
[<Element a at 0x1e1a27dfc78>]
15 건 저장완료..
[]
15 건 저장완료..
[]
15 건 저장완료..
[]
4 건 저장완료..
[]
[]


In [21]:
NEWS_LIST = db_select()

In [22]:
print(NEWS_LIST)

                                        URL  \
0    https://v.daum.net/v/20200505233813244   
1    https://v.daum.net/v/20200505221207681   
2    https://v.daum.net/v/20200505220039611   
3    https://v.daum.net/v/20200505213028369   
4    https://v.daum.net/v/20200505213024368   
..                                      ...   
344  https://v.daum.net/v/20200505050005064   
345  https://v.daum.net/v/20200505030230453   
346  https://v.daum.net/v/20200505030131395   
347  https://v.daum.net/v/20200505000136415   
348  https://v.daum.net/v/20200505000102404   

                                       TITLE  \
0                     "동물착취 말고 채식하세요" [포토뉴스]   
1    중국, 창정 5B 운반로켓 발사 성공.."우주정류장 건설 서막"(종합)   
2             APEC 통상장관, 경제활동 필수인력 이동 원활화 합의   
3                  사고난 차 사진 보내면 AI가 예상수리비 계산   
4                      LG의 스마트폰 부활, '벨벳'에 건다   
..                                       ...   
344    오늘은 어린이날..IPTV·케이블TV서 키즈 콘텐츠 '무료· 할인'   
345                 집콕족 늘면서 데이터 소비-5G 가입 '쑥'   
346

In [25]:
def save_excel(NEWS_LIST): #엑셀 파일 생성
    excel = pd.ExcelWriter('뉴스_IT_20200505.xlsx')
    NEWS_LIST.to_excel(excel,'.',index=False)
    excel.save()

In [26]:
save_excel(NEWS_LIST)

In [27]:
print(span)

[]
